In [1]:
import keras
import csv
import scipy
import numpy as np
import nltk #from nltk.tokenize import punkt #.tokenize.moses import MosesTokenizer
from gensim.models import KeyedVectors
from itertools import islice
import pickle
import pandas as pd
import re
from nltk.corpus import stopwords
import json
import pickle
from cyordereddict import OrderedDict
from joblib import Parallel, delayed

DATASET_FILE_PATH = "./DataSet/yelp.csv/yelp.csv"
WORD2VEC_FILE_PATH = "./DataSet/FBword2vec/wiki.en.vec"
TRAINING_DATA_PATH = "./DataSet/trainingData"
EXTENDED_DATASET_PATH = "./DataSet/review.json"

###File Names
TEST_NAME = "standard_tokenizer_No_Stopwords_noPunctuation_FB_vectors_100percentDataset"

EMBEDDED_MATRIX_NAME = TEST_NAME+'_embedded_matrix.pkl'
TOKENIZED_SENTENCES_NAME = TEST_NAME+ "_tokenized_sentences.pkl"
WORD_INDEX_NAME = TEST_NAME+ "_word_index.pkl"
INDEXED_TOKENIZED_SENTENCES_NAME = TEST_NAME + "_indexed_tokenized_sentences.pkl"

LABELS_NAME = 'labels.pkl'


AMOUNT_OF_WORDS = 70
FACEBOOK_EMBEDDING_SIZE = 300

C:\Users\skymont\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\skymont\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
word_index = pickle.load(open(WORD_INDEX_NAME,'rb'))

In [3]:
embedded_matrix =  np.load(EMBEDDED_MATRIX_NAME+'.npy')
print('embedded matrix loaded')

embedded matrix loaded


In [4]:
indexed_tokenized_sentences = pickle.load(open(INDEXED_TOKENIZED_SENTENCES_NAME,'rb'))

In [5]:
def reshapeSentences(indexed_tokenized_sentences,sentence_length):
    for i in range(len(indexed_tokenized_sentences)):
        padding = sentence_length - len(indexed_tokenized_sentences[i])
        indexed_tokenized_sentences[i] = indexed_tokenized_sentences[i][:sentence_length] + [-1]*padding
    return indexed_tokenized_sentences

In [6]:
### BUILDING MODEL

In [7]:
reshaped_Sentences = reshapeSentences(indexed_tokenized_sentences, AMOUNT_OF_WORDS)

In [8]:
labels = pd.read_pickle(LABELS_NAME)
labels.head()
labels.pop('cool')
labels.pop('funny')
labels.pop('useful')
labels.head()
labels = pd.get_dummies(labels['stars'])
labels.head()

,1,2,3,4,5
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,1,0


In [9]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import numpy as np
from keras import backend
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [10]:
embedding_layer = Embedding(
    len(word_index),
    FACEBOOK_EMBEDDING_SIZE,
    weights=[embedded_matrix],
    input_length=AMOUNT_OF_WORDS,
    trainable=True
)

In [11]:
reshaped_Sentences = np.stack(reshaped_Sentences,axis=0)

In [12]:

X_train, X_test, y_train, y_test = train_test_split(reshaped_Sentences, labels.values, test_size=0.2, random_state=42)

In [13]:
def generator(features, labels, batch_size, replace=True):
    
    batch_features = np.zeros((batch_size,AMOUNT_OF_WORDS))
    batch_labels = np.zeros((batch_size,5))
    while True:
        index = np.random.choice(features.shape[0], batch_size, replace=replace)
        batch_features[:] = features[index]
        batch_labels[:] = labels[index]
        yield batch_features, batch_labels
        

In [14]:
def percentageCorrect(y_true, y_pred):
    return backend.mean(backend.equal(y_true, backend.round(y_pred)))*100

In [22]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
SpatialDropout1D(0.5)
model.add(Conv1D(filters=16, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=8, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
SpatialDropout1D(0.5)
model.add(Conv1D(filters=8, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
SpatialDropout1D(0.5)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 70, 300)           57183900  
_________________________________________________________________
dropout_4 (Dropout)          (None, 70, 300)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 70, 32)            96032     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 35, 32)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 35, 16)            5136      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 17, 16)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 17, 8)             1288      
__________

In [28]:
#model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=40, batch_size=10, verbose=2)
model.fit_generator(generator(X_train,y_train,7000,False),steps_per_epoch=20, epochs=30, verbose=2, validation_data=generator(X_test, y_test, 15000, False), validation_steps=5)

Epoch 1/30
 - 7s - loss: 0.8460 - categorical_accuracy: 0.6450 - val_loss: 0.9251 - val_categorical_accuracy: 0.6317
Epoch 2/30
 - 6s - loss: 0.8414 - categorical_accuracy: 0.6496 - val_loss: 0.9741 - val_categorical_accuracy: 0.6134
Epoch 3/30
 - 6s - loss: 0.8383 - categorical_accuracy: 0.6482 - val_loss: 0.9694 - val_categorical_accuracy: 0.6132
Epoch 4/30
 - 6s - loss: 0.8375 - categorical_accuracy: 0.6474 - val_loss: 0.9695 - val_categorical_accuracy: 0.6089
Epoch 5/30
 - 6s - loss: 0.8366 - categorical_accuracy: 0.6499 - val_loss: 0.9515 - val_categorical_accuracy: 0.6202
Epoch 6/30
 - 6s - loss: 0.8376 - categorical_accuracy: 0.6486 - val_loss: 0.9528 - val_categorical_accuracy: 0.6263
Epoch 7/30
 - 6s - loss: 0.8365 - categorical_accuracy: 0.6491 - val_loss: 0.9351 - val_categorical_accuracy: 0.6337
Epoch 8/30
 - 6s - loss: 0.8334 - categorical_accuracy: 0.6503 - val_loss: 0.9419 - val_categorical_accuracy: 0.6303
Epoch 9/30
 - 6s - loss: 0.8373 - categorical_accuracy: 0.6487 -